In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from edgar import Edgartools
from edgar.filing import Filing
from datetime import date
from typing import List, Dict, Any

COMPANIES: List[str] = ["AAPL", "MSFT", "NVDA", "TSLA"]
FORM_TYPE: str = "10-K"
START_DATE: date = date(2024, 1, 1) # 2022年1月1日以降
USER_AGENT_EMAIL: str = "your.email@example.com"

# Edgartoolsの初期化
try:
    sec_data = Edgartools(user_agent=USER_AGENT_EMAIL)
except Exception as e:
    print(f"Edgartoolsの初期化中にエラーが発生しました: {e}")
    exit()

results: List[Dict[str, Any]] = []

# --- ファイルの一括検索・処理 ---
print(f"--- {COMPANIES} の最新の {FORM_TYPE} を取得・処理します ---")
for company_ticker in COMPANIES:
    print(f"\n[{company_ticker}] の {FORM_TYPE} を検索中...")

    try:
        # 1. フィルタリングされたFilingのリストを取得
        # 'limit=1' で最新の1件のみを取得
        # 'start_date' で期間を絞ることで、最新とは限らないが特定の期間内の最新を取得できる
        filings: List[Filing] = sec_data.get_filings(
            ticker_cik=company_ticker,
            form_types=[FORM_TYPE],
            start_date=START_DATE
        ).limit(1)

        if not filings:
            print(f"  -> {company_ticker} の条件に一致する {FORM_TYPE} は見つかりませんでした。")
            continue

        # 2. 最新のFilingオブジェクトを取得
        latest_filing: Filing = filings[0]

        print(f"  -> 提出日: {latest_filing.filing_date}, URL: {latest_filing.report_url}")

        # 3. Filingの内容オブジェクト（tenkオブジェクト）を取得
        # この処理は時間がかかる場合があります（ダウンロードとパースのため）
        tenk_obj = latest_filing.obj()

        # 4. MD&Aセクションを抽出
        # 10-Kの場合、この属性（management_discussion）は利用可能です
        md_and_a_text = tenk_obj.management_discussion

        # 5. 結果をリストに追加
        results.append({
            'company': company_ticker,
            'filing_date': latest_filing.filing_date,
            'report_url': latest_filing.report_url,
            # MD&Aのテキストを保存 (テキストが非常に長いため、文字数を取得)
            'md_and_a_length': len(md_and_a_text) if md_and_a_text else 0,
            'md_and_a_text_preview': md_and_a_text[:500] + "..." if md_and_a_text else "N/A",
        })

    except Exception as e:
        print(f"  -> [{company_ticker}] の処理中にエラーが発生しました: {e}")
        # エラーが発生した場合も、次の企業へ進む
        results.append({
            'company': company_ticker,
            'filing_date': 'Error',
            'report_url': 'Error',
            'md_and_a_length': 0,
            'md_and_a_text_preview': f"処理エラー: {e}",
        })
        continue

# --- 3. 結果の整理 ---
if results:
    final_df = pd.DataFrame(results)

    print("\n--- 全ての企業の MD&A 抽出結果 ---")
    print(final_df[['company', 'filing_date', 'md_and_a_length', 'report_url']])

    # テキスト全体を含めてCSVに保存したい場合は、以下の行を有効にします
    # final_df.to_csv(f"tenk_md_and_a_summary_{date.today()}.csv", index=False)
    # print(f"\n✅ 結果概要を 'tenk_md_and_a_summary_{date.today()}.csv' に保存しました。")
else:
    print("\n⚠️ 取得条件に一致する Filing は見つかりませんでした。設定を確認してください。")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


c:\Users\Yuki Hata\anaconda3\envs\py313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'Edgartools' from 'edgar' (c:\Users\Yuki Hata\anaconda3\envs\py313\Lib\site-packages\edgar\__init__.py)

## Financials の取得


In [ ]:
apple = Company("AAPL")
print(apple.name)

financials = apple.get_quarterly_financials()
income_statement = financials.income_statement().to_dataframe()
display(income_statement)

balance_sheet = financials.balance_sheet().to_dataframe()
display(balance_sheet)


Apple Inc.


,concept,label,2025-06-28,2024-06-29,level,abstract,dimension,balance,weight,preferred_sign
0,us-gaap_IncomeStatementAbstract,Income Statement [Abstract],NaN,NaN,0,True,False,NaN,NaN,NaN
1,us-gaap_StatementTable,Statement [Table],NaN,NaN,1,True,False,NaN,NaN,NaN
2,srt_ProductOrServiceAxis,Product and Service [Axis],NaN,NaN,2,True,False,NaN,NaN,NaN
3,srt_ProductsAndServicesDomain,Product and Service [Domain],NaN,NaN,3,True,False,NaN,NaN,NaN
4,us-gaap_ProductMember,Products,NaN,NaN,4,False,False,NaN,NaN,NaN
5,us-gaap_ServiceMember,Services,NaN,NaN,4,False,False,NaN,NaN,NaN
6,us-gaap_StatementLineItems,Statement [Line Items],NaN,NaN,2,True,False,NaN,NaN,NaN
7,us-gaap_RevenueFromContractWithCustomerExcludi...,Contract Revenue,3.136950e+11,2.961050e+11,3,False,False,credit,1.0,1.0
8,us-gaap_RevenueFromContractWithCustomerExcludi...,Products,2.332870e+11,2.249080e+11,4,False,True,credit,1.0,1.0
9,us-gaap_RevenueFromContractWithCustomerExcludi...,Services,8.040800e+10,7.119700e+10,4,False,True,credit,1.0,1.0


,concept,label,2025-06-28,2024-09-28,level,abstract,dimension,balance,weight,preferred_sign
0,us-gaap_StatementOfFinancialPositionAbstract,Statement of Financial Position [Abstract],None,None,0,True,False,NaN,NaN,NaN
1,us-gaap_AssetsAbstract,ASSETS:,None,None,1,True,False,NaN,NaN,NaN
2,us-gaap_AssetsCurrentAbstract,Current assets:,None,None,2,True,False,NaN,NaN,NaN
3,us-gaap_CashAndCashEquivalentsAtCarryingValue,Cash and Cash Equivalents,36269000000.0,29943000000.0,3,False,False,debit,1.0,1.0
4,us-gaap_MarketableSecuritiesCurrent,Current Marketable Securities,19103000000.0,35228000000.0,3,False,False,None,1.0,1.0
5,us-gaap_AccountsReceivableNetCurrent,Accounts Receivable,27557000000.0,33410000000.0,3,False,False,debit,1.0,1.0
6,us-gaap_NontradeReceivablesCurrent,Vendor non-trade receivables,19278000000.0,32833000000.0,3,False,False,None,1.0,1.0
7,us-gaap_InventoryNet,Inventory,5925000000.0,7286000000.0,3,False,False,debit,1.0,1.0
8,us-gaap_OtherAssetsCurrent,Other Current Assets,14359000000.0,14287000000.0,3,False,False,debit,1.0,1.0
9,us-gaap_AssetsCurrent,Total Current Assets,122491000000.0,152987000000.0,3,False,False,debit,1.0,1.0


In [ ]:
tenk = apple.get_filings(form="10-K").latest().obj()
financials = tenk.financials
income_statement = financials.income_statement
print(income_statement)


<bound method Financials.income_statement of ╭───────────────────────────────────────────────── XBRL Document ─────────────────────────────────────────────────╮
│ Apple Inc. (AAPL) • CIK 0000320193                                                                              │
│                                                                                                                 │
│          Form:  10-K                                                                                            │
│ Fiscal Period:  Fiscal Year 2025 (ended Sep 27, 2025)                                                           │
│          Data:  1,131 facts • 182 contexts                                                                      │
│                                                                                                                 │
│ Available Data Coverage:                                                                                        │
│   Annual: FY 2025, FY 202

## Filings


In [ ]:
company = Company("MSFT")
filing = company.get_filings(form="10-K").latest()
print(filing.company_name)
tenk = filing.obj()
print(tenk)

md_and_a = tenk.management_discussion
print(type(md_and_a))
print(md_and_a)


AttributeError: 'EntityFiling' object has no attribute 'company_name'

In [ ]:
"""
複数企業・複数期間でSEC Filingsを取得するコード
対応フォーム: 10-K, 10-Q, 8-K
"""

from edgar import *
import pandas as pd
from datetime import datetime

# ユーザー情報の設定（SEC規制により必須）
set_identity("youxitiankaggle.com")  # 実際のメールアドレスに変更すること

# === 設定パラメータ ===
COMPANIES = ["AAPL", "MSFT", "NVDA", "GOOGL", "TSLA"]
FORM_TYPES = ["10-K", "10-Q", "8-K"]
START_DATE = "2022-01-01"
END_DATE = "2024-12-31"
LIMIT_PER_FORM = 5  # フォームタイプごとの最大取得件数

# === Filingsの一括取得 ===
all_filings_data = []

print(f"=== {len(COMPANIES)}社の SEC Filings を取得開始 ===")
print(f"期間: {START_DATE} ~ {END_DATE}")
print(f"対象フォーム: {', '.join(FORM_TYPES)}\n")

for ticker in COMPANIES:
    print(f"[{ticker}] のFilingsを取得中...")

    try:
        company = Company(ticker)

        for form_type in FORM_TYPES:
            filings = company.get_filings(form=form_type).filter(
                date=f"{START_DATE}:{END_DATE}"
            )

            count = 0
            for filing in filings:
                if count >= LIMIT_PER_FORM:
                    break

                filing_info = {
                    "ticker": ticker,
                    "company_name": company.name,
                    "form_type": filing.form,
                    "filing_date": filing.filing_date,
                    "accession_number": filing.accession_no,
                    "filing_url": filing.homepage_url,  # 修正: homepage_url を使用
                }

                all_filings_data.append(filing_info)
                count += 1

            print(f"  {form_type}: {count}件取得")

    except Exception as e:
        print(f"  ❌ エラー: {e}")
        continue

print(f"\n=== 取得完了: 合計 {len(all_filings_data)}件 ===\n")

# === 結果をDataFrameに変換 ===
filings_df = pd.DataFrame(all_filings_data)

if not filings_df.empty:
    filings_df = filings_df.sort_values(
        ["ticker", "filing_date"], ascending=[True, False]
    )

    print("=== 取得データのサマリー ===")
    print(f"\n企業別の取得件数:")
    print(filings_df.groupby("ticker").size())

    print(f"\nフォームタイプ別の取得件数:")
    print(filings_df.groupby("form_type").size())

    print(f"\n最初の10件:")
    display(filings_df.head(10))

    # CSVに保存（オプション）
    # output_file = f"sec_filings_{datetime.now().strftime('%Y%m%d')}.csv"
    # filings_df.to_csv(output_file, index=False)
    # print(f"\n✅ データを {output_file} に保存しました")
else:
    print("⚠️ データが取得できませんでした")


=== 5社の SEC Filings を取得開始 ===
期間: 2022-01-01 ~ 2024-12-31
対象フォーム: 10-K, 10-Q, 8-K

[AAPL] のFilingsを取得中...
  10-K: 3件取得
  10-Q: 5件取得
  8-K: 5件取得
[MSFT] のFilingsを取得中...
  10-K: 3件取得
  10-Q: 5件取得
  8-K: 5件取得
[NVDA] のFilingsを取得中...
  10-K: 3件取得
  10-Q: 5件取得
  8-K: 5件取得
[GOOGL] のFilingsを取得中...
  10-K: 3件取得
  10-Q: 5件取得
  8-K: 5件取得
[TSLA] のFilingsを取得中...
  10-K: 4件取得
  10-Q: 5件取得
  8-K: 5件取得

=== 取得完了: 合計 66件 ===

=== 取得データのサマリー ===

企業別の取得件数:
ticker
AAPL     13
GOOGL    13
MSFT     13
NVDA     13
TSLA     14
dtype: int64

フォームタイプ別の取得件数:
form_type
10-K      15
10-K/A     1
10-Q      25
8-K       25
dtype: int64

最初の10件:


,ticker,company_name,form_type,filing_date,accession_number,filing_url
0,AAPL,Apple Inc.,10-K,2024-11-01,0000320193-24-000123,https://www.sec.gov/Archives/edgar/data/320193...
8,AAPL,Apple Inc.,8-K,2024-10-31,0000320193-24-000120,https://www.sec.gov/Archives/edgar/data/320193...
9,AAPL,Apple Inc.,8-K,2024-09-10,0001140361-24-040659,https://www.sec.gov/Archives/edgar/data/320193...
10,AAPL,Apple Inc.,8-K,2024-08-26,0001140361-24-038601,https://www.sec.gov/Archives/edgar/data/320193...
11,AAPL,Apple Inc.,8-K,2024-08-23,0001140361-24-038403,https://www.sec.gov/Archives/edgar/data/320193...
3,AAPL,Apple Inc.,10-Q,2024-08-02,0000320193-24-000081,https://www.sec.gov/Archives/edgar/data/320193...
12,AAPL,Apple Inc.,8-K,2024-08-01,0000320193-24-000080,https://www.sec.gov/Archives/edgar/data/320193...
4,AAPL,Apple Inc.,10-Q,2024-05-03,0000320193-24-000069,https://www.sec.gov/Archives/edgar/data/320193...
5,AAPL,Apple Inc.,10-Q,2024-02-02,0000320193-24-000006,https://www.sec.gov/Archives/edgar/data/320193...
1,AAPL,Apple Inc.,10-K,2023-11-03,0000320193-23-000106,https://www.sec.gov/Archives/edgar/data/320193...


In [ ]:
"""
複数企業の10-Kから特定のItem（セクション）を一括取得
"""

COMPANIES_BATCH = ["AAPL", "MSFT", "NVDA", "GOOGL"]
items_results = []

print("=== 複数企業の10-K Itemsを一括取得 ===\n")

for ticker in COMPANIES_BATCH:
    print(f"[{ticker}] を処理中...")

    try:
        company = Company(ticker)
        latest_10k = company.get_filings(form="10-K").latest()

        print(f"  提出日: {latest_10k.filing_date}")

        # 10-Kオブジェクトを取得
        tenk = latest_10k.obj()

        # Item 1: Business
        business_text = tenk.business if hasattr(tenk, "business") else None

        # Item 1A: Risk Factors
        risk_text = tenk.risk_factors if hasattr(tenk, "risk_factors") else None

        # Item 7: MD&A
        mda_text = (
            tenk.management_discussion
            if hasattr(tenk, "management_discussion")
            else None
        )

        result = {
            "ticker": ticker,
            "company_name": company.name,
            "filing_date": latest_10k.filing_date,
            "business_length": len(business_text) if business_text else 0,
            "risk_factors_length": len(risk_text) if risk_text else 0,
            "mda_length": len(mda_text) if mda_text else 0,
            "business_text": business_text,
            "risk_factors_text": risk_text,
            "mda_text": mda_text,
            "url": latest_10k.homepage_url,
        }

        items_results.append(result)
        print(
            f"  ✓ 完了 (Business: {result['business_length']:,}文字, Risk: {result['risk_factors_length']:,}文字, MD&A: {result['mda_length']:,}文字)\n"
        )

    except Exception as e:
        print(f"  ❌ エラー: {e}\n")
        continue

# 結果をDataFrameに変換
items_batch_df = pd.DataFrame(items_results)

print("\n=== 取得結果のサマリー ===")
display(
    items_batch_df[
        [
            "ticker",
            "company_name",
            "filing_date",
            "business_length",
            "risk_factors_length",
            "mda_length",
        ]
    ]
)

# 特定企業のテキストを確認（例：AAPL）
if not items_batch_df.empty:
    print("\n=== AAPL のBusiness セクションのプレビュー ===")
    aapl_row = items_batch_df[items_batch_df["ticker"] == "AAPL"]
    if not aapl_row.empty and aapl_row.iloc[0]["business_text"]:
        business_preview = aapl_row.iloc[0]["business_text"][:500]
        print(business_preview + "...")

# CSVに保存する場合（テキスト列を除外してサマリーのみ保存）
# summary_df = items_batch_df[['ticker', 'company_name', 'filing_date', 'business_length', 'risk_factors_length', 'mda_length', 'url']]
# summary_df.to_csv(f"10k_items_summary_{datetime.now().strftime('%Y%m%d')}.csv", index=False)
# print(f"\n✅ サマリーをCSVに保存しました")


## 複数企業の 10-K から特定 Item を一括取得

複数企業の 10-K から Item 1 (Business)と Item 1A (Risk Factors)を一括で取得する。


In [ ]:
"""
AAPLの最新10-Kから各Item（セクション）のテキストを取得する
"""

from edgar import Company

# AAPLのCompanyオブジェクトを取得
apple = Company("AAPL")

# 最新の10-Kを取得
latest_10k_filing = apple.get_filings(form="10-K").latest()

print(f"企業名: {apple.name}")
print(f"フォーム: {latest_10k_filing.form}")
print(f"提出日: {latest_10k_filing.filing_date}")
print(f"Accession Number: {latest_10k_filing.accession_no}")
print(f"URL: {latest_10k_filing.homepage_url}\n")

# 10-Kオブジェクトを取得（パース処理のため時間がかかる）
print("10-Kを解析中...")
tenk = latest_10k_filing.obj()

print("\n=== 利用可能な主要セクション（Items） ===\n")

# 各Itemのテキストを取得して表示
items_data = []

# Item 1: Business
if hasattr(tenk, "business"):
    business_text = tenk.business
    print(f"✓ Item 1 - Business")
    print(f"  文字数: {len(business_text):,}")
    print(f"  プレビュー: {business_text[:150]}...\n")

    items_data.append(
        {
            "item": "Item 1 - Business",
            "text_length": len(business_text),
            "text": business_text,
        }
    )

# Item 1A: Risk Factors
if hasattr(tenk, "risk_factors"):
    risk_text = tenk.risk_factors
    print(f"✓ Item 1A - Risk Factors")
    print(f"  文字数: {len(risk_text):,}")
    print(f"  プレビュー: {risk_text[:150]}...\n")

    items_data.append(
        {
            "item": "Item 1A - Risk Factors",
            "text_length": len(risk_text),
            "text": risk_text,
        }
    )

# Item 7: Management's Discussion and Analysis (MD&A)
if hasattr(tenk, "management_discussion"):
    mda_text = tenk.management_discussion
    print(f"✓ Item 7 - Management's Discussion and Analysis")
    print(f"  文字数: {len(mda_text):,}")
    print(f"  プレビュー: {mda_text[:150]}...\n")

    items_data.append(
        {"item": "Item 7 - MD&A", "text_length": len(mda_text), "text": mda_text}
    )

# すべての利用可能な属性を確認（デバッグ用）
print("\n=== 10-Kオブジェクトの全属性 ===")
available_attrs = [attr for attr in dir(tenk) if not attr.startswith("_")]
print(f"利用可能な属性数: {len(available_attrs)}")
print(f"主な属性: {available_attrs[:20]}")

# 結果をDataFrameで整理
items_df = pd.DataFrame(items_data)
print("\n=== 取得したItemsのサマリー ===")
display(items_df[["item", "text_length"]])


企業名: Apple Inc.
フォーム: 10-K
提出日: 2025-10-31
Accession Number: 0000320193-25-000079
URL: https://www.sec.gov/Archives/edgar/data/320193/0000320193-25-000079-index.html

10-Kを解析中...

=== 利用可能な主要セクション（Items） ===

✓ Item 1 - Business
  文字数: 16,004
  プレビュー: Item 1.    Business
Company Background
The Company designs, manufactures and markets smartphones, personal computers, tablets, wearables and accessori...

✓ Item 1A - Risk Factors
  文字数: 68,069
  プレビュー: Item 1A.    Risk Factors
The following summarizes factors that could have a material adverse effect on the Company’s business, reputation, results of ...

✓ Item 7 - Management's Discussion and Analysis
  文字数: 21,009
  プレビュー: Item 7.    Management’s Discussion and Analysis of Financial Condition and Results of Operations
The following discussion should be read in conjunctio...


=== 10-Kオブジェクトの全属性 ===
利用可能な属性数: 20
主な属性: ['balance_sheet', 'business', 'cash_flow_statement', 'chunked_document', 'company', 'directors_officers_and_governance', 

,item,text_length
0,Item 1 - Business,16004
1,Item 1A - Risk Factors,68069
2,Item 7 - MD&A,21009


## 10-K の各 Item（セクション）のテキストを取得

10-K は以下の主要な Item で構成される：

-   Item 1: Business（事業内容）
-   Item 1A: Risk Factors（リスク要因）
-   Item 7: Management's Discussion and Analysis (MD&A)
-   Item 8: Financial Statements（財務諸表）
-   その他の Item
